# K-Map Visualizer
This Jupyter notebook allows users to see how a kmap can be solved.

Simply input the matrix you want and press play.

In [1]:
from manim import *

import numpy as np
config.media_width = "100%"
_RV = "-v WARNING -qm --progress_bar None --disable_caching Example"

In [2]:
from pyeda.inter import *
import re

In [3]:
a, b, c, d = map(exprvar, 'abcd')

input_equation = Or(~a & ~b & c, a & d, b & ~d, c & ~d, a & ~c, ~a & ~d)
f = espresso_exprs(input_equation)[0]
f


Or(a, ~d, And(~b, c))

In [4]:
simplified_input = str(f)[3:][:-1]
comma_regex = r',\s*(?![^()]*\))'
simplified_input = re.split(comma_regex, simplified_input)
simplified_input


simplified_input  

tt = expr2truthtable(f)



def graycode_gen(n: int):
    """Generate Gray code ordering for n bits."""
    result = [i ^ (i >> 1) for i in range(2**n)]
    return result

def truth_table_to_kmap(truth_table):
    # Size can be odd like for a 3 table.
    num_vars = len(truth_table.support)
    half_vars = num_vars // 2
    num_rows = 2**half_vars
    num_cols = 2**(num_vars - half_vars)
    
    row_graycode, col_graycode = graycode_gen(half_vars), graycode_gen(num_vars - half_vars)
    
    kmap = np.zeros((num_rows, num_cols), dtype=int)
    
    for position, value in truth_table.iter_relation():
      position = tuple(position.values())
      row_index = row_graycode[position[:half_vars][0] * 2 + position[1]]
      col_index = col_graycode[position[half_vars:][0] * 2 + position[3]]
      kmap[row_index, col_index] = value
    return kmap

truth_table_to_kmap(tt)



array([[1, 0, 1, 1],
       [1, 0, 0, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1]])

In [5]:
terms = f.support
input_matrix = np.matrix([[0] * len(terms) for _ in range(len(terms))])
input_matrix

matrix([[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]])

## Settings:

In [6]:
# Settings
input_matrix = truth_table_to_kmap(tt)
   
   
predefined_colors = [PURPLE, MAROON, RED, ORANGE, GOLD, YELLOW, GREEN, TEAL, BLUE]

In [7]:
# Adds colors from the solved groups      
import kmap_animator
mingroups = kmap_animator.create_min_groups(input_matrix)
mop_mingroup = []
color_picker = kmap_animator.color_selector(predefined_colors)
for mingroup_cluster in mingroups:
  selected_color = color_picker()
  for mingroup in mingroup_cluster:
    mop_mingroup.append({"coords": mingroup, "color": selected_color})

## Animation Output:

In [8]:
%%manim $_RV
# Animation manim magic to see
class Example(Scene):
    def construct(self):
        # Example matrix (you can replace this with any n x m matrix)
        matrix = input_matrix

        # Call the function to generate the K-map
        kmap, labels = self.create_kmap(matrix)

        # Add a title
        title = Text("Karnaugh Map", font_size=24).to_edge(UP)
        self.play(Write(title))

        # Add the K-map labels and grid to the scene
        self.play(Create(labels), Create(kmap))


        self.wait(1)
        
        # Define rectangles using top-left and bottom-right coordinates
        rectangles = mop_mingroup

        # Get the top-left corner of the K-map
        kmap_corner = kmap.get_corner(UL)  # UL = Upper Left in Manim
        kmap_bottom_right = kmap.get_corner(DR)  # DR = Down Right in Manim

        # Constants for cell dimensions
        cell_width = 1.0
        cell_height = 1.0
        cell_margin = 0
        corner_radius = 0.5
        dashes = 10


        # Add encompassing rectangle around the entire K-map
        kmap_width = kmap_bottom_right[0] - kmap_corner[0]
        kmap_height = kmap_corner[1] - kmap_bottom_right[1]
        kmap_rect = Rectangle(
            width=kmap_width,
            height=kmap_height,
            color=WHITE,
            fill_opacity=0.0,  # No fill, only border
             stroke_opacity=0.1,
        )
        kmap_rect.move_to(kmap.get_center())
        
        dashed_kmap_rect = DashedVMobject(kmap_rect, num_dashes=40)


        
        self.play(Create(kmap_rect))

        # Draw the rectangles with alpha transparency, clipped within the K-map boundaries
        for i, rect in enumerate(rectangles):
            # Unpack the given coordinates
            (coord1, coord2) = rect["coords"]
            color = rect["color"]

            # Calculate the most encompassing rectangle's corners
            top_left_row = min(coord1[1], coord2[1])
            top_left_col = min(coord1[0], coord2[0])
            bottom_right_row = max(coord1[1], coord2[1])
            bottom_right_col = max(coord1[0], coord2[0])

            # Calculate rectangle coordinates
            top_left_x = kmap_corner[0] + top_left_col * (cell_width)
            top_left_y = kmap_corner[1] - top_left_row * (cell_height)
            bottom_right_x = kmap_corner[0] + (bottom_right_col + 1) * (cell_width)
            bottom_right_y = kmap_corner[1] - (bottom_right_row + 1) * (cell_height)


            # Recalculate width, height, and center after clipping
            width = bottom_right_x - top_left_x + cell_margin 
            height = top_left_y - bottom_right_y + cell_margin 
            center_x = (top_left_x + bottom_right_x) / 2
            center_y = (top_left_y + bottom_right_y) / 2

            # Create the filled rectangle with transparency (alpha=0.2)
            rectangle = RoundedRectangle(
                corner_radius=min(corner_radius, width / 2, height / 2),
                width=width,
                height=height,
                color=color,
                stroke_opacity=0.3,
                fill_opacity=0.3  # Set transparency
            )
            
            rectangle.move_to([center_x, center_y, 0])
            clipped_rectangle = Intersection(rectangle, kmap_rect, color=color, fill_opacity = 0.1, stroke_opacity=0.3)
            # Add the rectangle to the scene
            self.play(Create(clipped_rectangle), run_time=0.5)
            

        self.wait(3)

    def create_kmap(self, matrix):
        """
        Create a flat Karnaugh Map representation for an n x m matrix with Gray code labels.
        """
        n = len(matrix)      # Number of rows
        m = len(matrix[0])   # Number of columns

        # Constants for layout
        cell_width = 1.0
        cell_height = 1.0
        cell_margin = 0

        # Group to hold all K-map elements
        kmap_group = VGroup()
        labels_group = VGroup()

        # Gray code generation for labels
        def gray_code(num_bits):
            return [bin(i ^ (i >> 1))[2:].zfill(num_bits) for i in range(2 ** num_bits)]

        row_labels = gray_code(int.bit_length(max(1, n - 1)))
        col_labels = gray_code(int.bit_length(max(1, m - 1)))

        get_color = lambda x: YELLOW if x == 1 else GRAY
        get_bold  = lambda x: NORMAL if x == 1 else NORMAL
        
        # Generate the grid of the K-map
        for i, row in enumerate(matrix):
            for j, value in enumerate(row):
                # Create the rectangle for the cell
                rect = Rectangle(width=cell_width, height=cell_height, color=GRAY,  stroke_opacity=0.1)
                rect.move_to(np.array([(j - m / 2) * (cell_width + cell_margin) + cell_width / 2,
                                       -(i - n / 2) * (cell_height + cell_margin) - cell_height / 2,
                                       0]))

                # Add the value as a text in the center of the cell
          
                text = Text(str(value), font_size=36, color=get_color(value), weight=get_bold(value))
                text.set_z_index(1)
                text.move_to(rect.get_center())

                # Group the rectangle and text together
                cell_group = VGroup(rect, text)

                # Add to the overall K-map group
                kmap_group.add(cell_group)

        def add_table_labels(idx, label, labelType: str):
            label_text = Text(label, font_size=24)
            is_col = labelType == "COL"
            label_text.next_to(kmap_group[0].get_top() if is_col else kmap_group[0].get_left() , 
                               UP if is_col else LEFT, 
                               buff=0.5)
            label_text.shift((RIGHT if is_col else DOWN) * idx * 
                             (cell_width if is_col else cell_height + cell_margin))
            labels_group.add(label_text)
        
        # Add row labels
        for i, label in enumerate(row_labels):
            add_table_labels(i, label, "ROW")

        # Add column labels
        for j, label in enumerate(col_labels):
            add_table_labels(j, label, "COL")

        return kmap_group, labels_group

Manim Community v0.18.1